<a href="https://colab.research.google.com/github/ThomasAbeyta/AudioPollutionMeter/blob/main/Kidd_Fire_Weather_Station_WindRose_Plots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Plotting Weather Station Data

**Objection**

 This code was written to read data from a CSV file, filters it based on time range and station, and creates either a combined chart or individual charts using Plotly Express. The resulting charts are displayed or saved as PNG images, then calculates the hourly average of specific columns within a given time range, filters the results, and saves them to a CSV file. Finally the function returns the filtered DataFrame.


- The create_charts function takes several parameters: csv_path (path to the CSV file), time_range (a tuple specifying the start and end time for filtering data), station_list (a list of station names), output_dir (the directory to save the output charts), and combine (a boolean flag indicating whether to create a combined chart or individual charts for each station).

 1. The function starts by reading the data from the CSV file specified by csv_path. It selects specific columns ('Station', 'Date', 'Time', 'Wind_Src_deg', 'Speed_mph', and 'Gust_mph') using the usecols parameter in the pd.read_csv function. The data is loaded into a DataFrame called df.

 2. The 'Date' and 'Time' columns are combined into a single 'DateTime' column by converting them to strings and concatenating them. The resulting column is assigned to df['DateTime'] using the .loc indexer and pd.to_datetime function.

 3. The data is filtered based on the specified time range using the .between method on the 'DateTime' column. The filtered data is stored in the filtered_df DataFrame, which is created as a copy of the filtered data to avoid the SettingWithCopyWarning.

 4. The 'Wind_flipped' column is created by adding 180 to the 'Wind_Src_deg' column and applying the modulus operator % with 360. This calculation flips the wind source degrees by adding 180 and taking the remainder after dividing by 360.

 5. If combine is True, a combined chart is created. The function filters the filtered_df DataFrame based on the stations in the station_list using the .isin method. The resulting DataFrame is assigned to combined_df.

 6. The combined chart is created using Plotly Express (px.bar_polar). The 'Speed_mph' column is mapped to the radial axis (r), the 'Wind_flipped' column is mapped to the angular axis (theta), and the 'Speed_mph' column is used for coloring the bars (color). Various styling options to create PNG files using fig.update_layout.

 7. The combined chart is displayed using fig.show() and saved as a PNG image using fig.write_image. The image is saved in the output_dir directory with the filename "combined_chart.png". Finally, the function returns the combined_df DataFrame.

 8. If combine is False, individual charts are created for each station in the station_list. The function iterates over each station and filters the filtered_df DataFrame for that station. The station-specific chart is created and displayed using the same steps as in the combined chart section.

 9. These charts are displayed using fig.show() and saved as a PNG image using fig.write_image. The image is saved in the output_dir directory with the filename "combined_chart.png". 

 10. Then the filtered_df is used to generate an average of two columns, 'Speed_mph' and 'Wind_Src_deg', within a specified time range.

 11. The start_date and end_date are extracted from the time_range list.

 12. The time_diff variable is calculated by subtracting the converted end_date from the converted start_date using pd.to_datetime. This represents the time difference between the start and end dates.

 13. The speed_hourly and wind_hourly DataFrames are created by grouping the station_df DataFrame by the DateTime column using pd.Grouper with the frequency set to time_diff. The agg function is used to calculate the mean for the respective columns.

 14. The column hierarchy of the resulting DataFrames is flattened by renaming the columns using a list comprehension and the '_'.join(col).strip() operation.

 15. The two DataFrames, speed_hourly and wind_hourly, are concatenated horizontally along with the 'Speed_mph_mean' column using pd.concat.
 
 16. The resulting DataFrame is filtered based on the start_date and end_date by creating a boolean mask using the 'DateTime_' column (renamed from 'DateTime' in the previous steps).

 17. The filtered DataFrame is saved to a CSV file using the to_csv function, with the filename constructed based on the station, start_date, and end_date, and the index set to False.

 18. Finally, the function returns the filtered_df DataFrame.

# Data Source

#Date range from 03/17/2022 - 03/18/2022

KTXGORMA19
Station Name:Living in the Weather Bubble
Latitude / Longitude:32.329° N, 98.906° W
Elevation:1424
City:Cisco
State:TX
Hardware:Tycon Power Systems ProWeatherStation
Software:EasyWeatherV1.6.4
https://www.wunderground.com/dashboard/pws/KTXGORMA19/graph/2022-03-18/2022-03-18/daily


KTXCISCO13
Station Name:ERS
Latitude / Longitude: 32.33 °N, 98.91 °W
Elevation:1578
City:Cisco
State:TX
Hardware:Tycon Power Systems ProWeatherStation
Software:EasyWeatherV1.6.4
https://www.wunderground.com/dashboard/pws/KTXCISCO13/graph/2022-03-17/2022-03-17/daily


KTXEASTL14
Station Name:Moss Oak Farm
Latitude / Longitude: 32.40 °N, 98.82 °W
Elevation:1453
City:Cisco
State:TX
Hardware:Tycon Power Systems ProWeatherStation
Software:EasyWeatherV1.6.4
https://www.wunderground.com/dashboard/pws/KTXEASTL14/table/2022-03-17/2022-03-17/daily

KTXEASTL20
Station Name:Lake Leon SRC
Latitude / Longitude: 32.38 °N, 98.71 °W
Elevation:1467
City:Cisco
State:TX
Hardware:Tycon Power Systems ProWeatherStation
Software:EasyWeatherV1.6.4
https://www.wunderground.com/dashboard/pws/KTXEASTL20/table/2022-03-17/2022-03-17/daily

KTXRISIN9
Station Name:A&R Ranch
Latitude / Longitude: 32.02 °N, 98.90 °W
Elevation:1870
City:Cisco
State:TX
Hardware:Tycon Power Systems ProWeatherStation
Software:EasyWeatherV1.6.4
https://www.wunderground.com/dashboard/pws/KTXRISIN9/table/2022-03-17/2022-03-17/daily




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Installs

In [ ]:
pip install -U kaleido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.7 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

#WindRose Function

In [ ]:
def create_charts(csv_path, time_range, station_list, output_dir, combine=True):
  # Read data from csv
  df = pd.read_csv(csv_path, header=0, 
                   usecols = ['Station', 'Date', 'Time', 'Wind_Src_deg', 'Speed_mph', 'Gust_mph'])

  # Format DateTime column
  df.loc[:,'DateTime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'].astype(str).replace('/', '-'))

  # Filter data within specified time range
  filtered_df = df[df['DateTime'].between(time_range[0], time_range[1])].copy()

  # Flip wind source degrees
  filtered_df['Wind_flipped'] = (filtered_df['Wind_Src_deg'] + 180) % 360

  if combine:
    # Filter for the stations in the list
    combined_df = filtered_df[filtered_df['Station'].isin(station_list)]

    # Create combined chart for all stations within the time range
    fig = px.bar_polar(combined_df, r='Speed_mph', theta='Wind_flipped', color='Speed_mph',
                       template='plotly+presentation', 
                       color_discrete_sequence= px.colors.sequential.Plasma_r,
                       range_color=[0,25])
    fig.update_layout(
            # Background colors and other visual options
            paper_bgcolor="rgba(0,0,0,0)",
            plot_bgcolor="rgba(0,0,0,0)",
            showlegend=False,
            coloraxis_showscale=False,
            polar=dict(
              bgcolor="rgba(0,0,0,0)", 
              radialaxis=dict(
                  gridcolor="rgba(0,0,0,0)",  # radial axis color to transparent
                  linecolor="rgba(0,0,0,0)",   # radial line color to transparent
                  tickfont=dict(color="rgba(0,0,0,0)"),  # Make radial axis numbers transparent
                  
                  ),
                  angularaxis=dict(
                      gridcolor="rgba(0,0,0,0)",  # angular axis color to transparent
                      linecolor="rgba(0,0,0,0)",   # angular line color to transparent
                      tickfont=dict(color="rgba(0,0,0,0)"),  # Make radial axis numbers transparent
                      
                      )
                  ),
                  font=dict(
                      color="rgba(0,0,0,0)"  # This makes all the font transparent
                  )
        )
    fig.show()
    # Save figure as PNG
    fig.write_image(os.path.join(output_dir, "combined_chart.png"))
    return combined_df
  else:
    # Create and save a chart for each station
    for station in station_list:
      station_df = filtered_df[filtered_df['Station'] == station]
      
      fig = px.bar_polar(station_df, r='Speed_mph', theta='Wind_flipped', color='Speed_mph',
                         template='plotly+presentation', 
                         color_discrete_sequence= px.colors.sequential.Plasma_r,
                         range_color=[0,25])
      fig.update_layout(
            # Background colors and other visual options
            paper_bgcolor="rgba(0,0,0,0)",
            plot_bgcolor="rgba(0,0,0,0)",
            showlegend=False,
            coloraxis_showscale=False,
            polar=dict(
              bgcolor="rgba(0,0,0,0)", 
              radialaxis=dict(
                  gridcolor="rgba(0,0,0,0)",  # radial axis color to transparent
                  linecolor="rgba(0,0,0,0)",   # radial line color to transparent
                  tickfont=dict(color="rgba(0,0,0,0)"),  # Make radial axis numbers transparent
                  
                  ),
                  angularaxis=dict(
                      gridcolor="rgba(0,0,0,0)",  # angular axis color to transparent
                      linecolor="rgba(0,0,0,0)",   # angular line color to transparent
                      tickfont=dict(color="rgba(0,0,0,0)"),  # Make radial axis numbers transparent
                      
                      )
                  ),
                  font=dict(
                      color="rgba(0,0,0,0)"  # This makes all the font transparent
                  )
        )
      fig.show()
      #Save fig as PNG
      fig.write_image(os.path.join(output_dir, f"{station}_{time_range[0]}_to_{time_range[1]}.png"))
      
      # Group by hour and calculate mean
      start_date, end_date = time_range
      time_diff = pd.to_datetime(end_date) - pd.to_datetime(start_date)

      speed_hourly = station_df.set_index('DateTime').groupby(pd.Grouper(freq=time_diff)).agg({'Speed_mph': ['mean']}).reset_index()
      wind_hourly = station_df.set_index('DateTime').groupby(pd.Grouper(freq=time_diff)).agg({'Wind_Src_deg': ['mean']}).reset_index()

      # Flatten the column hierarchy
      speed_hourly.columns = ['_'.join(col).strip() for col in speed_hourly.columns.values]
      wind_hourly.columns = ['_'.join(col).strip() for col in wind_hourly.columns.values]

      # Concatenate the two dataframes
      hourly_avg = pd.concat([wind_hourly, speed_hourly['Speed_mph_mean']], axis=1)

      # Filter the DataFrame
      mask = (hourly_avg['DateTime_'] >= start_date) & (hourly_avg['DateTime_'] <= end_date)
      hourly_avg = hourly_avg.loc[mask]

      # Save hourly means to CSV
      hourly_avg.to_csv(os.path.join(output_dir, f"{station}_{time_range[0]}_to_{time_range[1]}_hourly_avg.csv"), index=False)
     
  return filtered_df
  
station_list = ['KTXGORMA19','KTXCISCO13','KTXEASTL14','KTXEASTL20','KTXRISIN9']
time_range = ('2022-03-17 16:00:00', '2022-03-17 20:00:00')

# Toms csv path and output
csv_path = '/content/drive/MyDrive/Billable Work/Kidd_fire/Weather Stations/All_Weather_Data.csv'
output_dir = '/content/drive/MyDrive/Billable Work/Kidd_fire/Weather Stations/'

# Jesse's csv path and output
# csv_path = '/content/drive/Shareddrives/Customers/ONCOR/Kidd Fire/Notebooks/All_Weather_Data.csv'
# output_dir = '/path/to/output/directory' 


create_charts(csv_path, time_range, station_list, output_dir, combine=False)

,Station,Date,Time,Wind_Src_deg,Speed_mph,Gust_mph,DateTime,Wind_flipped
192,KTXGORMA19,03/17/2022,16:04:00,225.0,12.1,15.0,2022-03-17 16:04:00,45.0
193,KTXGORMA19,03/17/2022,16:09:00,225.0,15.8,19.5,2022-03-17 16:09:00,45.0
194,KTXGORMA19,03/17/2022,16:14:00,225.0,10.9,13.6,2022-03-17 16:14:00,45.0
195,KTXGORMA19,03/17/2022,16:19:00,247.5,13.4,16.3,2022-03-17 16:19:00,67.5
196,KTXGORMA19,03/17/2022,16:24:00,247.5,12.5,15.6,2022-03-17 16:24:00,67.5
...,...,...,...,...,...,...,...,...
3078,KTXRISIN9,03/17/2022,19:39:00,270.0,9.7,11.4,2022-03-17 19:39:00,90.0
3079,KTXRISIN9,03/17/2022,19:44:00,270.0,9.4,12.7,2022-03-17 19:44:00,90.0
3080,KTXRISIN9,03/17/2022,19:49:00,270.0,7.6,9.0,2022-03-17 19:49:00,90.0
3081,KTXRISIN9,03/17/2022,19:54:00,270.0,7.7,9.3,2022-03-17 19:54:00,90.0
